In [8]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

import pandas as pd
from src.GGUFBot import GGUFBot
from src.AnswerGenerator import AnswerGenerator
import json
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
bot = GGUFBot(model_path="/home/hatakeyama/python/ChatServer/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf", 
              max_new_tokens=4000, n_ctx=4000, n_gpu_layers=400)
#bot = GGUFBot(args.model_path, max_new_tokens=args.max_new_tokens, n_ctx=args.max_new_tokens, n_gpu_layers=args.n_layers)



llama_model_loader: additional 3 GGUFs metadata loaded.
llama_model_loader: loaded meta data with 29 key-value pairs and 563 tensors from /home/hatakeyama/python/ChatServer/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models--mistralai--Mixtral-8x22B-Inst...
llama_model_loader: - kv   2:                          llama.block_count u32              = 56
llama_model_loader: - kv   3:                       llama.context_length u32              = 65536
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 6144
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 16384
llama_model_loader: - 

llama_model_loader: - kv  17:                      tokenizer.ggml.scores arr[f32,32768]   = [-1000.000000, -1000.000000, -1000.00...
llama_model_loader: - kv  18:                  tokenizer.ggml.token_type arr[i32,32768]   = [3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
llama_model_loader: - kv  19:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  20:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  21:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  22:               tokenizer.ggml.add_bos_token bool             = false
llama_model_loader: - kv  23:               tokenizer.ggml.add_eos_token bool             = false
llama_model_loader: - kv  24:                    tokenizer.chat_template str              = {{bos_token}}{% for message in messag...
llama_model_loader: - kv  25:               general.quantization_version u32              = 2
llama_model_loader: - kv  26:

loading model...


ggml_cuda_init: GGML_CUDA_FORCE_MMQ:   yes
ggml_cuda_init: CUDA_USE_TENSOR_CORES: no
ggml_cuda_init: found 8 CUDA devices:
  Device 0: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 1: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 2: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 3: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 4: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 5: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 6: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 7: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
llm_load_tensors: ggml ctx size =    2.51 MiB
llm_load_tensors: offloading 56 repeating layers to GPU
llm_load_tensors: offloading non-repeating layers to GPU
llm_load_tensors: offloaded 57/57 layers to GPU
llm_load_tensors:        CPU buffer size =   132.00 MiB
llm_load_tensors:      CUDA0 buffer size = 12238.73 MiB
llm_load_tenso

In [10]:

a_gen = AnswerGenerator(bot,n_answers=1)

In [11]:
df=pd.DataFrame()

temp_df=pd.read_csv("data/raw_qa/0426spread.csv")
df=pd.concat([df,temp_df])


path_list=[
    "data/raw_qa/t1.jsonl",
    "data/raw_qa/t2.jsonl",
]
for path in path_list:
    temp_df= pd.read_json(path, orient='records', lines=True)
    df=pd.concat([df,temp_df])


In [12]:
records=df.to_dict(orient='records')

In [13]:
out_path="data/completion_records.jsonl"

if os.path.exists(out_path):
    done_df=pd.read_json(out_path, orient='records', lines=True)
    done_questions=list(done_df["question"].values)
else:
    done_questions=[]


In [14]:
import copy

for record in records:
    if record["question"] in done_questions:
        print(f"skip {record['question']}")
        continue
    a_gen(record)
    record["answer_1"]=copy.copy(record["answer"])
    with open(out_path, 'a') as f:
        f.write(json.dumps(record, ensure_ascii=False)+'\n')

skip こんにちは､日本語喋れますか?
skip 日本語で､なにか楽しいことを話してください



llama_print_timings:        load time =    3183.72 ms
llama_print_timings:      sample time =      61.05 ms /   236 runs   (    0.26 ms per token,  3865.68 tokens per second)
llama_print_timings: prompt eval time =    3183.56 ms /    41 tokens (   77.65 ms per token,    12.88 tokens per second)
llama_print_timings:        eval time =   13488.20 ms /   235 runs   (   57.40 ms per token,    17.42 tokens per second)
llama_print_timings:       total time =   17849.30 ms /   276 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3183.72 ms
llama_print_timings:      sample time =      86.21 ms /   379 runs   (    0.23 ms per token,  4396.29 tokens per second)
llama_print_timings: prompt eval time =    1211.75 ms /    36 tokens (   33.66 ms per token,    29.71 tokens per second)
llama_print_timings:        eval time =   23953.11 ms /   378 runs   (   63.37 ms per token,    15.78 tokens per second)
llama_print_timings:       total time =   26202.57 ms /   414 

In [ ]:
record.pop("answer")

with open(out_path, 'w') as f:
    f.write(json.dumps({"a":"b"}, ensure_ascii=False)+'\n')

In [ ]:
    with open(out_path, 'a') as f:
        f.write(json.dumps(record, ensure_ascii=False)+'\n')

ValueError: Circular reference detected

In [ ]:
record

{'question': 'こんにちは､日本語喋れますか?',
 'answer': {...},
 'inst_answer_0': '日本語で簡潔に回答してください: こんにちは､日本語喋れますか?',
 'answer_0': ' はい、当然のことながら日本語を話せます。何かお手伝いできることはありますか？',
 'answer_1': {'question': 'こんにちは､日本語喋れますか?',
  'answer': {...},
  'inst_answer_0': '日本語で簡潔に回答してください: こんにちは､日本語喋れますか?',
  'answer_0': ' はい、当然のことながら日本語を話せます。何かお手伝いできることはありますか？'}}